In [ ]:
# -*- coding: utf-8 -*-
from __future__ import print_function, unicode_literals
import requests
import pymongo
import time
import datetime
import random

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [ ]:
connection = pymongo.MongoClient("mongodb://localhost")
ershou_col = connection.cq.lianjia_ershou_details

In [ ]:
# 装载所有id
ershou_set = set()
for i in ershou_col.find({},{"_id":1}):
    ershou_set.add(i["_id"])

In [ ]:
driver = webdriver.Chrome('/Users/levenls/Downloads/chromedriver')
driver.get("http://cq.lianjia.com/ershoufang/106091601340.html")

In [ ]:
sp_pattern = re.compile("<span.+span>")
brackets_ptr = re.compile("\(.+\)")
id_pattern = re.compile(r"/[0-9]+.html")
base_url = "https://cq.lianjia.com/ershoufang/pg{0}/"

In [ ]:
def get_house_details(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "lxml")
    h_d = {}
    h_d["_id"] = re.search(id_pattern, url).group(0).replace("/", "").replace(".html", "")
    h_d["title"] = soup.find("h1")["title"]
    h_d["desc"] = soup.find("div", class_="sub")["title"]
    h_d["total_price"] = float(soup.find("div", class_="price").find("span", class_="total").get_text())
    h_d["unit_price"] = soup.find("div", class_="unitPrice").get_text()
    h_d["tax"] = soup.find("span", class_="taxtext").get_text()
    base_info = soup.find("div", class_="base").find_all("li")
    for i in base_info:
        tmp = i.find("span").get_text().strip()
        if tmp == u"\u6240\u5728\u697c\u5c42":
            h_d["floor"] = re.sub(brackets_ptr, "", i.get_text().replace(tmp, "")).strip()
            h_d["total_floor"] = re.search(brackets_ptr, i.get_text().replace(tmp, "")).group(0).replace("(", "").replace(")","")
            continue
        h_d[tmp] = i.get_text().replace(tmp, "").strip()
        
    trade_info = soup.find("div", class_="transaction").find_all("li")
    for i in trade_info:
        tmp = i.find("span").get_text().strip()
        h_d[tmp] = i.get_text().replace(tmp, "").strip()
        
    h_d["area"] = ",".join([i.get_text() for i in soup.find("div", class_="areaName").find_all("a")])
    h_d["community_id"] = soup.find("div", class_="communityName").find("a")["href"].split("/")[-2]
    h_d["community_name"] = soup.find("div", class_="communityName").find("a").get_text()
    
    ex_info = soup.find_all("div", class_="introContent")[1].find_all("div", class_="baseattribute")
    for i in ex_info:
        tmp = i.find("div", class_="name").get_text().replace("\n", "").strip()
        h_d[tmp] = i.find("div", class_="content").get_text().replace("\n", "").strip()

    h_d["total_viewed"] = int(soup.find("div", class_="totalCount").find("span").get_text())
    ershou_col.insert_one(h_d)
    print ("录入了一套房: {0}".format(h_d["title"]))

In [ ]:
for i in log_progress(range(1, 101)):
    time.sleep(random.uniform(1.5,3.5))
    driver.get(base_url.format(i))
    rsoup = BeautifulSoup(driver.page_source, "lxml")
    for c in rsoup.find("ul", class_="sellListContent"):
        house_url = c.find("a")["href"]
        house_id = re.search(id_pattern, house_url).group(0).replace("/", "").replace(".html", "")

        # record_view
        record_view = {}
        folinfo = c.find("div", class_="followInfo").get_text()
        record_view["interested"] = folinfo.split("/")[0].strip().replace(u"\u4eba\u5173\u6ce8", "")
        record_view["viewed"] = folinfo.split("/")[1].strip().replace(u"\u5171","").replace(u"\u6b21\u5e26\u770b","")
        record_view["post_date"] = folinfo.split("/")[2].strip()
        record_view["spider_data"] = datetime.datetime.utcnow()
        view_col.insert_one(record_view)

        if house_id not in ershou_set:
            get_house_details(house_url)
            ershou_set.add(house_id)
            time.sleep(random.uniform(1.5,3.5))
        else:
            print ("已存在数据库，跳过抓取")